## 爬取url

In [1]:
#把分類跟 url 組合在一起
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import os
import datetime
import time
import math
urls = "" #裝要印出的所有url
main_url ='http://www.chinatimes.com'

#時間
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y-%m-%d')
fileName = yestoday.replace('-','')

#target url
page_url = 'http://www.chinatimes.com/history-by-date/{}-2601'.format(yestoday)

#request submit
res = requests.get(page_url)
soup = BeautifulSoup(res.text)

#今天有多少篇新聞
div = soup.select('.listLeft li')[0]
totalNewsNum = div.select('span')[1].text.replace('(','').replace(')','')

#無條件進位，算出總共會有幾頁新聞
totalPageNum = int(math.ceil(int(totalNewsNum) / 11))
# print(totalPageNum)

#爬取每一頁的11篇新聞
for n in range(1,totalPageNum+1):
    #target url
    page_url = 'http://www.chinatimes.com/history-by-date/{}-2601?page={}'.format(yestoday,n)
    
    #request submit
    res = requests.get(page_url)
    soup = BeautifulSoup(res.text)
    
    #抓到每一頁11篇新聞的url
    lis = soup.select('.listRight li')
    
    #sleep
    time.sleep(3)
    
    for li in lis:
        href = li.select('a')[0]['href']
        category = li.select('a')[1].text.strip()
        news_url = urljoin(main_url,href)+'\n' #urljoin會自動合併兩個網址

        #進行新聞的篩選
        if(category != '藝文副刊' and category != '《旺來報》' and category != '時尚消費'):
            urls += category + news_url  #把新聞串在一起等著印出
#             print(category)
#             print(news_url)       
f = open('C:/Users/ytchen/Desktop/ETtoday/{}.txt'.format(fileName),'w')
f.write(urls.encode('utf-8')) #最後再一次寫出來
f.close()  

C:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: UnicodeWarning: Unicode unequal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


## 將新聞爬取下來

In [ ]:
#這個才是最正確的
#修改成一次爬好幾個月
import string
import datetime as dt
import requests
import time
from bs4 import BeautifulSoup
import json
import time

#全域變數
error_count = 0
error_url = ''
all_json_str = [] #存所有資訊的 json

#抓日期
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y-%m-%d')
fileName = yestoday.replace('-','')

#開起每天的url檔案
f = open('C:/Users/ytchen/Desktop/ETtoday/{}.txt'.format(fileName),'r')
for url_no in f.readlines():
    
    try:
        #抓類別
        category = url_no.split('http')[0]

        #因為text檔每行後面有換行符號\n，必須要去除掉，否則會沒辦法送出request
        url = 'http'+ url_no.split('http')[1].replace('\n','')   #重新組合url   
        res = requests.get(url) #利用get發送請求
        soup = BeautifulSoup(res.text)

        #抓標題跟副標題
        article = soup.select('article')[0]
        title = article.select('h1')[0].text
        # print(title)

        #點閱率
        div = soup.select('.art_click.clear-fix')[0]
        click = div.select('span')[1].text
        # print(click)

        #抓內文
        article = ''
        div= soup.select('article')[0]   #先把所有內文抓到
        ps = div.select('p')                #再把裡面所有的p標籤抓到
        for p in ps:                        #把每個p標籤一個一個抓出來印
            article += p.text.replace('\r','').replace('\n','')  #段落會隱藏換行符號\r，要取代掉，不然後面讀取json格式會只讀到最後一行
        # print(article)

        #抓keyword
        keyNum = len(soup.select('.a_k a'))
        allKeyword = ''
        for n in range(0,keyNum):
            keyword = soup.select('.a_k a')[n].text
            allKeyword += keyword +' '
        allKeyword = allKeyword.strip()
        # print(allKeyword)

        #將資訊存成 json檔
        data  =  { 
            'comp':'ettoday',  #新聞來源
            'category':category, #類別
            'title':title, #標題
            'content':article, #內文
            'date':fileName,  #直接抓檔案的日期來用
            'url':url,
            'keyw':allKeyword,
            'click':click
        }

        #一篇新聞一本字典
        all_json_str.append(data) 
        time.sleep(2)
    except:
        error_count +=1
        error_url += url+','
        continue         
#檔案儲存
f1 = open('C:/Users/ytchen/Desktop/ETtoday/json/{}.json'.format(fileName),'w')
b =json.dumps(all_json_str) #再把 list 字典轉換成 json 並包起來
f1.write(b)

#寫入檔案結束
f1.close()
f.close()

#log記錄
logString = ''
fileName_log = fileName+'/'+'errorCount:'+str(error_count)+'\n'+'error_url:'+error_url+'\n'
#讀取
with open('C:/Users/ytchen/Desktop/ETtoday/log_content.txt','r') as f1:
    for logLine in f1.readlines():
        logString += logLine
#重新寫入
with open('C:/Users/ytchen/Desktop/ETtoday/log_content.txt','w') as f2:
    logString += fileName_log
    f2.write(logString.encode('utf-8'))

## 將json倒入MongoDB

In [1]:
import pymongo
from pymongo import MongoClient
import json
import datetime
today = datetime.datetime.now()
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y-%m-%d')
fileName = yestoday.replace('-','')
# fileName = '20160917'

f = open('C:/Users/ytchen/Desktop/ETtoday/json/{}.json'.format(fileName),'r')
i = f.read()
json = json.loads(i)
print(len(json))
    
client = MongoClient() 
database = client["test"]
collection = database["news"]
collection.insert_many(
    json
)
client.close()
f.close()

99
